## Load data

In [17]:
# use this command when cannot run cuda
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [18]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [19]:
# x_train, y_train, x_test, y_test = x_train[:5000], y_train[:5000], x_test[:1000], y_test[:1000]

In [20]:
x_train.shape, x_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [21]:
y_train.shape

(50000, 1)

In [22]:
import numpy as np
for label in range(10):
    print("No. {}: {}".format(label, np.sum(y_train==label)))

No. 0: 5000
No. 1: 5000
No. 2: 5000
No. 3: 5000
No. 4: 5000
No. 5: 5000
No. 6: 5000
No. 7: 5000
No. 8: 5000
No. 9: 5000


In [23]:
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train, num_classes=10)
y_test_one_hot = to_categorical(y_test, num_classes=10)
y_train_one_hot.shape

(50000, 10)

## Self-built model

In [24]:
from keras.models import Sequential
from keras.applications import VGG16
from keras.layers import Conv2D, MaxPooling2D, Input, Flatten, Dense

In [25]:
# base_model = VGG16(include_top=False, input_shape=(32,32,3), weights='imagenet')
# for layer in base_model.layers:
#     block, type = layer.name.split('_')
#     if block == 'block5' and type != 'pool':
#         layer.trainable = True
#     else:
#         layer.trainable = False
#     # layer.trainable = False
#     # print(layer.name, layer.trainable)

In [26]:
# model = Sequential()
# model.add(base_model)
# model.add(Flatten())
# model.add(Dense(units=64, activation='relu'))
# model.add(Dense(units=10, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# model.summary()

In [31]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(32,32,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 15, 15, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_19 (Conv2D)          (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_17 (MaxPooli  (None, 6, 6, 64)          0         
 ng2D)                                                           
                                                                 
 conv2d_20 (Conv2D)          (None, 4, 4, 128)         73856     
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 2, 2, 128)        

In [32]:
model.fit(x_train, y_train_one_hot,
          validation_data=(x_test, y_test_one_hot),
          epochs=10,
          batch_size=32)

Epoch 1/10
1563/1563 [==============================] - 11s 6ms/step - loss: 1.8623 - accuracy: 0.3799 - val_loss: 1.3697 - val_accuracy: 0.5140
Epoch 2/10
1563/1563 [==============================] - 8s 5ms/step - loss: 1.3385 - accuracy: 0.5350 - val_loss: 1.5106 - val_accuracy: 0.4850
Epoch 3/10
1563/1563 [==============================] - 8s 5ms/step - loss: 1.1993 - accuracy: 0.5913 - val_loss: 1.5109 - val_accuracy: 0.5074
Epoch 4/10
1563/1563 [==============================] - 9s 6ms/step - loss: 1.1122 - accuracy: 0.6238 - val_loss: 1.3109 - val_accuracy: 0.5622
Epoch 5/10
1563/1563 [==============================] - 9s 5ms/step - loss: 1.0452 - accuracy: 0.6482 - val_loss: 1.2176 - val_accuracy: 0.6009
Epoch 6/10
1563/1563 [==============================] - 8s 5ms/step - loss: 1.0080 - accuracy: 0.6654 - val_loss: 1.2328 - val_accuracy: 0.6035
Epoch 7/10
1563/1563 [==============================] - 9s 6ms/step - loss: 0.9778 - accuracy: 0.6789 - val_loss: 1.1357 - val_accuracy

In [40]:
preds = (model.predict(x_test[10:20]))
for pred in preds:
    print(np.argmax(pred), end = ' ')

1/1 [==============================] - 0s 32ms/step
8 9 3 7 9 8 5 2 8 6 

In [41]:
y_test[10:20]

array([[0],
       [9],
       [5],
       [7],
       [9],
       [8],
       [5],
       [7],
       [8],
       [6]], dtype=uint8)

## Pretrained model

In [63]:
from keras.models import Sequential
from keras.applications import VGG16
from keras.layers import Conv2D, MaxPooling2D, Input, Flatten, Dense

In [64]:
base_model = VGG16(include_top=False, input_shape=(32,32,3), weights='imagenet')
for layer in base_model.layers:
    # block, type = layer.name.split('_')
    # if block == 'block5':# and type != 'pool':
    #     layer.trainable = True
    # else:
    #     layer.trainable = False
    layer.trainable = False
    # print(layer.name, layer.trainable)

In [65]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_7 (Flatten)         (None, 512)               0         
                                                                 
 dense_14 (Dense)            (None, 64)                32832     
                                                                 
 dense_15 (Dense)            (None, 10)                650       
                                                                 
Total params: 14748170 (56.26 MB)
Trainable params: 33482 (130.79 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [66]:
# model = Sequential()
# model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(32,32,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Flatten())
# model.add(Dense(units=64, activation='relu'))
# model.add(Dense(units=10, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# model.summary()

In [67]:
model.fit(x_train, y_train_one_hot,
          validation_data=(x_test, y_test_one_hot),
          epochs=10,
          batch_size=32)

Epoch 1/10
1563/1563 [==============================] - 18s 11ms/step - loss: 2.0886 - accuracy: 0.4465 - val_loss: 1.4670 - val_accuracy: 0.5224
Epoch 2/10
1563/1563 [==============================] - 17s 11ms/step - loss: 1.3564 - accuracy: 0.5587 - val_loss: 1.4055 - val_accuracy: 0.5613
Epoch 3/10
1563/1563 [==============================] - 16s 10ms/step - loss: 1.2711 - accuracy: 0.5913 - val_loss: 1.3613 - val_accuracy: 0.5711
Epoch 4/10
1563/1563 [==============================] - 17s 11ms/step - loss: 1.2321 - accuracy: 0.6056 - val_loss: 1.3822 - val_accuracy: 0.5801
Epoch 5/10
1563/1563 [==============================] - 19s 12ms/step - loss: 1.2129 - accuracy: 0.6120 - val_loss: 1.3799 - val_accuracy: 0.5829
Epoch 6/10
1563/1563 [==============================] - 16s 11ms/step - loss: 1.1916 - accuracy: 0.6200 - val_loss: 1.4489 - val_accuracy: 0.5829
Epoch 7/10
1563/1563 [==============================] - 16s 11ms/step - loss: 1.1796 - accuracy: 0.6233 - val_loss: 1.5202 -

In [68]:
preds = (model.predict(x_test[100:150]))
for pred in preds:
    print(np.argmax(pred), end = ' ')
print()
print(*(y_test[100:150].flatten()))

2/2 [==============================] - 0s 11ms/step
7 3 5 3 1 1 3 6 1 7 4 0 2 2 1 3 0 4 3 7 0 7 9 2 8 2 8 3 3 0 2 1 8 9 6 3 9 1 4 8 6 5 6 3 8 3 6 5 5 3 
4 5 6 3 1 1 3 6 8 7 4 0 6 2 1 3 0 4 2 7 8 3 1 2 8 0 8 3 5 2 4 1 8 9 1 2 9 7 2 9 6 5 6 3 8 7 6 2 5 2


## Pretrained model (trainable last block)

In [ ]:
from keras.models import Sequential
from keras.applications import VGG16
from keras.layers import Conv2D, MaxPooling2D, Input, Flatten, Dense

In [ ]:
base_model = VGG16(include_top=False, input_shape=(32,32,3), weights='imagenet')
for layer in base_model.layers:
    block, type = layer.name.split('_')
    if block == 'block5' and type != 'pool':
        layer.trainable = True
    else:
        layer.trainable = False
    # layer.trainable = False
    # print(layer.name, layer.trainable)

In [ ]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_5 (Flatten)         (None, 512)               0         
                                                                 
 dense_10 (Dense)            (None, 64)                32832     
                                                                 
 dense_11 (Dense)            (None, 10)                650       
                                                                 
Total params: 14748170 (56.26 MB)
Trainable params: 7112906 (27.13 MB)
Non-trainable params: 7635264 (29.13 MB)
_________________________________________________________________


In [ ]:
# model = Sequential()
# model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(32,32,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Flatten())
# model.add(Dense(units=64, activation='relu'))
# model.add(Dense(units=10, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# model.summary()

In [ ]:
model.fit(x_train, y_train_one_hot,
          validation_data=(x_test, y_test_one_hot),
          epochs=10,
          batch_size=32)

Epoch 1/10
1563/1563 [==============================] - 28s 17ms/step - loss: 2.3478 - accuracy: 0.5582 - val_loss: 1.2591 - val_accuracy: 0.5836
Epoch 2/10
1563/1563 [==============================] - 25s 16ms/step - loss: 1.0044 - accuracy: 0.6860 - val_loss: 1.0939 - val_accuracy: 0.6860
Epoch 3/10
1563/1563 [==============================] - 26s 17ms/step - loss: 0.9366 - accuracy: 0.7132 - val_loss: 0.9211 - val_accuracy: 0.6941
Epoch 4/10
1563/1563 [==============================] - 29s 18ms/step - loss: 0.9020 - accuracy: 0.7245 - val_loss: 1.0057 - val_accuracy: 0.6952
Epoch 5/10
1563/1563 [==============================] - 26s 16ms/step - loss: 0.8655 - accuracy: 0.7409 - val_loss: 0.8821 - val_accuracy: 0.7387
Epoch 6/10
1563/1563 [==============================] - 26s 16ms/step - loss: 0.8397 - accuracy: 0.7509 - val_loss: 0.9031 - val_accuracy: 0.7204
Epoch 7/10
1563/1563 [==============================] - 26s 16ms/step - loss: 0.8286 - accuracy: 0.7581 - val_loss: 0.9768 -

In [ ]:
preds = (model.predict(x_test[100:150]))
for pred in preds:
    print(np.argmax(pred), end = ' ')
print()
print(*(y_test[100:150].flatten()))

2/2 [==============================] - 1s 470ms/step
1 3 6 3 1 1 4 6 8 7 4 0 2 2 1 3 0 4 3 3 8 3 1 2 8 4 8 3 3 2 4 1 8 9 3 3 9 7 4 8 6 5 6 5 3 3 6 5 3 3 
4 5 6 3 1 1 3 6 8 7 4 0 6 2 1 3 0 4 2 7 8 3 1 2 8 0 8 3 5 2 4 1 8 9 1 2 9 7 2 9 6 5 6 3 8 7 6 2 5 2
